## Wheelchair Accessibility Analysis Pipeline

| Stage | Data/Process | Details |
|-------|-------------|----------|
| **Input from Simulation** | Physical Constraints | - Slope percentage (>5% flagged)<br>- Turn angles (>45° flagged)<br>- Path width (<1.2m flagged)<br>- Multiple turns in sequence<br>- Length of continuous slope |
| | Location Context | - Indoor/outdoor transition points<br>- Rest area availability<br>- Distance between challenges |
| | Temporal Data | - Total journey distance<br>- Number of stops required<br>- Number of direction changes |
| **Experience LLM Input** | Flagged Issues | Example: `{"slope": "8% for 10m", "turns": "90° turn after slope", "width": "1.1m at turn"}` |
| | Journey Sequence | Chronological sequence of challenges encountered |
| | Cumulative Impact | Combined effect of multiple challenges in sequence |
| **Experience LLM Output** | Physical Impact | - Energy expenditure assessment<br>- Fatigue point identification<br>- Required rest stops<br>- Physical strain evaluation |
| | Social Impact | - Where assistance might be needed<br>- Potential interaction challenges<br>- Independence implications |
| | Emotional Impact | - Stress points in journey<br>- Confidence impacts<br>- Dignity considerations |


In [21]:
from langchain_community.chat_models import ChatOpenAI
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain.chains import LLMChain

class AccessibilityAnalyzer:
    def __init__(self, openai_api_key):
        self.llm = ChatOpenAI(
            model_name="gpt-4",
            temperature=0.6,
            openai_api_key=openai_api_key
        )
        
        # standards must be changed to the actual standards...
        self.standards = {
            "corridor_width": {
                "min": 1.5,  
                "recommended": 2.0
            },
            "slope": {
                "max": 0.05,
                "recommended": 0.033
            },
            "turn_radius": {
                "min": 1.5,
                "recommended": 2.0
            },
            "rest_area_spacing": {
                "max": 30
            }
        }

    def check_violations(self, route_data):
        violations = []
        
        for segment in route_data["segments"]:
            if segment["width"] < self.standards["corridor_width"]["min"]:
                violations.append({
                    "type": "width_violation",
                    "location": segment["id"],
                    "actual": segment["width"],
                    "required": self.standards["corridor_width"]["min"],
                    "severity": "high"
                })
            
            if segment["slope"] > self.standards["slope"]["max"]:
                violations.append({
                    "type": "slope_violation",
                    "location": segment["id"],
                    "actual": segment["slope"],
                    "required": self.standards["slope"]["max"],
                    "severity": "high"
                })
            
            for turn in segment["turns"]:
                if turn["angle"] > 45:  
                    violations.append({
                        "type": "turn_violation",
                        "location": f"{segment['id']}_pos_{turn['position']}",
                        "actual": turn["angle"],
                        "severity": "medium"
                    })
            
            if "no_rest_area" in segment["features"]:
                if segment["length"] > self.standards["rest_area_spacing"]["max"]:
                    violations.append({
                        "type": "rest_area_violation",
                        "location": segment["id"],
                        "length": segment["length"],
                        "max_spacing": self.standards["rest_area_spacing"]["max"],
                        "severity": "medium"
                    })
        # need to add more based on the scenario..
        return violations


    def create_analysis_chain(self): # need to refine the prompts
        analysis_template = """
        Analyze the accessibility implications for wheelchair users based on the following data:
        
        Route Details: {route_data}
        
        Technical Violations Found: {violations}
        
        Provide your analysis in the following format:

        SOCIAL INTERACTION POINTS:
        1. [First point about required social interaction or assistance]
        2. [Second point]
        3. [Third point]

        DIGNITY IMPACTS:
        1. [First point about impact on dignity and independence]
        2. [Second point]
        3. [Third point]

        EMOTIONAL CONSIDERATIONS:
        1. [First point about emotional and psychological impact]
        2. [Second point]
        3. [Third point]

        Each point should be specific and concrete. Focus on the implications of the violations found.
        """
        
        analysis_prompt = ChatPromptTemplate.from_template(analysis_template)
        
        analysis_chain = LLMChain(
            llm=self.llm,
            prompt=analysis_prompt,
            output_key="analysis"
        )
        
        return analysis_chain

    def parse_llm_response(self, response):
        sections = {
            "social_interaction_points": [],
            "dignity_impacts": [],
            "emotional_considerations": []
        }
        
        current_section = None
        
        for line in response.split('\n'):
            line = line.strip()
            if not line:
                continue
                
            if "SOCIAL INTERACTION POINTS:" in line:
                current_section = "social_interaction_points"
                continue
            elif "DIGNITY IMPACTS:" in line:
                current_section = "dignity_impacts"
                continue
            elif "EMOTIONAL CONSIDERATIONS:" in line:
                current_section = "emotional_considerations"
                continue
                
            if current_section and line[0].isdigit():
                point = line.split('. ', 1)[1] if '. ' in line else line
                sections[current_section].append(point)
                
        return sections

    def analyze_route(self, route_data):
        if isinstance(route_data, str):
            import ast
            route_data = ast.literal_eval(route_data)
        
        violations = self.check_violations(route_data)
        results = self.create_analysis_chain().run({
            "route_data": route_data,
            "violations": violations
        })
        
        analysis = {
            "violations": violations,
            "analysis": self.parse_llm_response(results)
        }
        
        return analysis

In [22]:
route_data = {
    "segments": [
        {
            "id": "seg1",
            "type": "corridor",
            "length": 15,
            "width": 1.1,
            "slope": 0.08,
            "surface": "smooth",
            "turns": [{"angle": 90, "position": 10}],
            "context": "high_traffic_area",
            "features": ["no_passing_space", "no_rest_area"]
        }
    ],
    "environmental": {
        "time_of_day": "peak_hours",
        "lighting": "well_lit",
        "crowd_level": "high"
    }
}

analyzer = AccessibilityAnalyzer(openai_api_key)
results = analyzer.analyze_route(route_data)

print("\nAccessibility Violations:")
for violation in results["violations"]:
    print(f"\nViolation Type: {violation['type']}")
    print(f"Location: {violation['location']}")
    print(f"Severity: {violation['severity']}")
    if 'actual' in violation:
        print(f"Actual: {violation['actual']}")
    if 'required' in violation:
        print(f"Required: {violation['required']}")

for category, points in results["analysis"].items():
    print(f"\n{category.replace('_', ' ').title()}:")
    for point in points:
        print(f"- {point}")


Accessibility Violations:

Violation Type: width_violation
Location: seg1
Severity: high
Actual: 1.1
Required: 1.5

Violation Type: slope_violation
Location: seg1
Severity: high
Actual: 0.08
Required: 0.05

Violation Type: turn_violation
Location: seg1_pos_10
Severity: medium
Actual: 90

Social Interaction Points:
- The narrow width of the corridor (1.1m instead of the required 1.5m) might necessitate assistance from another person to avoid collision, particularly in the high traffic area. This can compromise the wheelchair user's independence.
- The high slope (0.08 instead of the maximum recommended 0.05) may require additional assistance to navigate, particularly for manual wheelchair users. This could lead to unwanted social interactions or reliance on others.
- The 90 degree turn at position 10 in the corridor could pose a challenge, potentially requiring assistance from others to execute safely, particularly given the high crowd level during peak hours.

Dignity Impacts:
- The l